In [ ]:
!pip install langchain transformers sentence_transformers

In [ ]:
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
# Загружаем модель и токенизатор
model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Проверяем, что модель загружена корректно
print("Модель загружена:", model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Модель загружена: sberbank-ai/rugpt3large_based_on_gpt2


In [ ]:
def generate_response(prompt, max_length=100, temperature=0.7, top_k=50, top_p=0.9, no_repeat_ngram_size=2, repetition_penalty=1.2, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_return_sequences=1,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        no_repeat_ngram_size=no_repeat_ngram_size,
        repetition_penalty=repetition_penalty,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        early_stopping=True
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Пример использования
prompt = "Привет, как ты?"
response = generate_response(prompt)
print("Ответ модели:\n", response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Ответ модели:
 Привет, как ты?

– Хорошо. А у тебя? – спросил он, но тут же понял, что это не так:
    я был весь в грязи и пыли; из-под одежды виднелись лишь грязные обноски.
 




In [ ]:
def neuro_employee_response(query):
    template = """
    Ты — консультант по управлению временем. Твоя задача — помогать пользователю планировать задачи, расставлять приоритеты и анализировать эффективность.

    Запрос пользователя: {query}

    Ответ консультанта:
    """
    prompt = template.format(query=query)
    return generate_response(prompt, max_new_tokens=50)  # Ограничиваем длину ответа

# Пример запроса
query = "Как мне распланировать день, если у меня много задач?"
response = neuro_employee_response(query)
print("Ответ нейро-сотрудника:\n", response)

Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Ответ нейро-сотрудника:
 
    Ты — консультант по управлению временем. Твоя задача — помогать пользователю планировать задачи, расставлять приоритеты и анализировать эффективность.

    Запрос пользователя: Как мне распланировать день, если у меня много задач?

    Ответ консультанта:
      В первую очередь нужно определить для себя сколько времени вы планируете уделять конкретной задаче или проекту (некоторые люди не могут выделить себе время на все подряд). Если хотите уделить ему больше всего своего внимания, то используйте «Заголовок дня».


In [ ]:
def filter_query(query):
    allowed_topics = ["планирование", "время", "задачи", "приоритеты"]
    if any(topic in query.lower() for topic in allowed_topics):
        return query
    else:
        return "Извините, я не могу ответить на этот вопрос."

# Пример использования фильтра
query = "Как мне построить машину времени?"
filtered_query = filter_query(query)
response = neuro_employee_response(filtered_query)
print("Ответ нейро-сотрудника:\n", response)

Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Ответ нейро-сотрудника:
 
    Ты — консультант по управлению временем. Твоя задача — помогать пользователю планировать задачи, расставлять приоритеты и анализировать эффективность.

    Запрос пользователя: Извините, я не могу ответить на этот вопрос.

    Ответ консультанта:
      
           Консультант может помочь Вам с планированием, если Вы укажете в запросе желаемую дату ответа или время его выполнения (заполнив форму запроса).
Как выбрать подарок для женщины? Подарок должен быть приятным! Но


In [ ]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=a9124168ae637ba291e70e9cc126aa5eace0f0f597bba26408aa17d34bae2219
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [ ]:
import wikipediaapi

# Указываем User-Agent
user_agent = "MyApp/1.0 (https://example.com; example@example.com)"

# Инициализация Wikipedia API с User-Agent
wiki_wiki = wikipediaapi.Wikipedia(
    language='ru',  # Указываем язык (русский)
    user_agent=user_agent  # Указываем User-Agent
)

# Загрузка статьи
page_title = "Управление_временем"
page = wiki_wiki.page(page_title)

# Проверка, что статья существует
if page.exists():
    print("Статья загружена:", page.title)
    article_text = page.text
else:
    print("Статья не найдена.")

Статья загружена: Управление временем


In [ ]:
import re

def clean_wikipedia_text(text):
    # Удаление заголовков вида "== Заголовок =="
    text = re.sub(r"==.*?==", "", text)

    # Удаление сносок вида "[1]", "[2]" и т.д.
    text = re.sub(r"\[\d+\]", "", text)

    # Удаление лишних пробелов и переносов строк
    text = re.sub(r"\n+", "\n", text).strip()

    # Разделение текста на абзацы
    paragraphs = text.split("\n")

    # Удаление пустых абзацев
    paragraphs = [p.strip() for p in paragraphs if p.strip()]

    return paragraphs

# Очистка текста статьи
cleaned_paragraphs = clean_wikipedia_text(article_text)

# Вывод первых нескольких абзацев
print("Обработанные абзацы:")
for p in cleaned_paragraphs[:5]:
    print(p)
    print("---")

Обработанные абзацы:
Управление временем, организация времени, тайм-менеджмент (англ. time management) — технология организации времени и повышения эффективности его использования.
---
Управление временем — это действие или процесс тренировки сознательного контроля над временем, потраченным на конкретные виды деятельности, при котором специально увеличиваются эффективность и продуктивность. Управление временем может помочь получить ряд навыков, инструментов и методов, используемых при выполнении конкретных задач, проектов и целей. Этот набор включает в себя широкий спектр деятельности, а именно: планирование, распределение, постановку целей, делегирование, анализ временных затрат, мониторинг, организацию, составление списков и расстановку приоритетов. Изначально управление приписывалось только бизнесу или трудовой деятельности, но со временем термин расширился, включив личную деятельность с таким же основанием. Система управления временем составляет сочетание процессов, инструментов, т

In [ ]:
def clean_wikipedia_text(text):
    # Удаление заголовков вида "== Заголовок =="
    text = re.sub(r"==.*?==", "", text)

    # Удаление сносок вида "[1]", "[2]" и т.д.
    text = re.sub(r"\[\d+\]", "", text)

    # Удаление лишних пробелов и переносов строк
    text = re.sub(r"\n+", "\n", text).strip()

    # Разделение текста на абзацы
    paragraphs = text.split("\n")

    # Удаление пустых абзацев
    paragraphs = [p.strip() for p in paragraphs if p.strip()]

    # Фильтрация абзацев по ключевым словам
    relevant_paragraphs = []
    for p in paragraphs:
        if "история" not in p.lower() and "россия" not in p.lower():
            relevant_paragraphs.append(p)

    return relevant_paragraphs

# Очистка текста статьи
cleaned_paragraphs = clean_wikipedia_text(article_text)

# Вывод первых нескольких абзацев
print("Обработанные абзацы:")
for p in cleaned_paragraphs[:5]:
    print(p)
    print("---")

Обработанные абзацы:
Управление временем, организация времени, тайм-менеджмент (англ. time management) — технология организации времени и повышения эффективности его использования.
---
Управление временем — это действие или процесс тренировки сознательного контроля над временем, потраченным на конкретные виды деятельности, при котором специально увеличиваются эффективность и продуктивность. Управление временем может помочь получить ряд навыков, инструментов и методов, используемых при выполнении конкретных задач, проектов и целей. Этот набор включает в себя широкий спектр деятельности, а именно: планирование, распределение, постановку целей, делегирование, анализ временных затрат, мониторинг, организацию, составление списков и расстановку приоритетов. Изначально управление приписывалось только бизнесу или трудовой деятельности, но со временем термин расширился, включив личную деятельность с таким же основанием. Система управления временем составляет сочетание процессов, инструментов, т

In [ ]:
!pip install -U langchain-community
!pip install faiss-gpu
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Создаем embedding-модель
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Создаем векторную базу данных
vectorstore = FAISS.from_texts(cleaned_paragraphs, embeddings)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
# Создаем pipeline для работы с моделью
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,  # Используем GPU, если доступно
    max_length=100,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=2,
    do_sample=True
)

# Интегрируем pipeline в LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Проверяем работу модели
print(llm("Привет, как ты?"))

Device set to use cuda:0
<ipython-input-24-261e8a576e7b>:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
<ipython-input-24-261e8a576e7b>:22: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm("Привет, как ты?"))
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select

Привет, как ты?

— Нормально, — ответила она, и я увидел, что она улыбается. Ты что-нибудь ел? Или хочешь пить? А где твой друг? Он сказал, чтобы я пришла к нему, но не сказал куда.
 В конце концов, она ушла, а я остался сидеть и смотреть на нее. Я видел, каким прекрасным было ее лицо, когда она улыбалась мне. И я не мог отвести от него глаз. Что-


In [ ]:
# Создаем pipeline для работы с моделью
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,  # Используем GPU, если доступно
    max_new_tokens=50,  # Ограничиваем длину выходных данных
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=2,
    do_sample=True
)

# Интегрируем pipeline в LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Проверяем работу модели
print(llm("Привет, как ты?"))

Device set to use cuda:0


Привет, как ты? Как самочувствие?

— Нормально, — ответила она. А я вот не знаю, что делать с собой.
 В глазах у нее стояли слезы. Она протянула мне носовой платок. Я взял его. И тут увидел


In [ ]:
# Интегрируем RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

# Пример запроса
query = "Какие методы управления временем существуют?"
response = qa_chain.invoke({"query": query})  # Используем метод invoke
print("Ответ с использованием базы знаний:\n", response)

Ответ с использованием базы знаний:
 {'query': 'Какие методы управления временем существуют?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nСлабая мотивация труда, приводящая к низкой производительности (зарплата давно не увеличивалась, долгое время нет повышения в должности), порождающей хронический недостаток времени.\n\nВ настоящее время существует множество школ тайм-менеджмента. Классическое обучение проводится по следующим темам:\n\nЛичная миссия как эмоционально мотивирующий фактор.\n\nоценить задачи соответственно по категориям A, B и C;\n\nQuestion: Какие методы управления временем существуют?\nHelpful Answer: 1. Планирование. 2. Регулирование времени (планирование). 3. Интенсивная оценка времени\n4. Сортировка времени по приоритетам. 5. Переключение внимания с задач на задачи. 6. Разбор задач"}


In [ ]:
from langchain.prompts import PromptTemplate

# Создаем собственный шаблон для RetrievalQA
prompt_template = """Используй следующие фрагменты контекста, чтобы ответить на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь, не пытайся придумать ответ.

{context}

Вопрос: {question}
Полезный ответ:"""

# Создаем PromptTemplate
custom_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Интегрируем RetrievalQA с настроенным шаблоном
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": custom_prompt}
)

In [ ]:
# Пример запроса
query = "Какие методы управления временем существуют?"
response = qa_chain.invoke({"query": query})  # Используем метод invoke

# Извлекаем только поле "result"
result = response["result"]
print("Ответ с использованием базы знаний:\n", result)

Ответ с использованием базы знаний:
 Используй следующие фрагменты контекста, чтобы ответить на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь, не пытайся придумать ответ.

Слабая мотивация труда, приводящая к низкой производительности (зарплата давно не увеличивалась, долгое время нет повышения в должности), порождающей хронический недостаток времени.

В настоящее время существует множество школ тайм-менеджмента. Классическое обучение проводится по следующим темам:

Личная миссия как эмоционально мотивирующий фактор.

оценить задачи соответственно по категориям A, B и C;

Вопрос: Какие методы управления временем существуют?
Полезный ответ: В настоящее в мире существует много методов. В этой книге я расскажу о том, как использовать все имеющиеся инструменты управления. Но для начала, пожалуйста, ответь на следующий вопрос: «Какие методы организации времени существуют в настоящее?»
Задавая вопрос


In [ ]:
from langchain.prompts import PromptTemplate

# Создаем упрощенный шаблон для RetrievalQA
prompt_template = """Используй следующие фрагменты контекста, чтобы ответить на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь.

{context}

Вопрос: {question}
Полезный ответ:"""

# Создаем PromptTemplate
custom_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Интегрируем RetrievalQA с настроенным шаблоном
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": custom_prompt}
)

In [ ]:
# Создаем pipeline для работы с моделью
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,  # Используем GPU, если доступно
    max_new_tokens=50,  # Ограничиваем длину выходных данных
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=2,
    do_sample=True
)

# Интегрируем pipeline в LangChain
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [ ]:
# Пример запроса
query = "Какие методы управления временем существуют?"
response = qa_chain.invoke({"query": query})  # Используем метод invoke

# Извлекаем только поле "result"
result = response["result"]
print("Ответ с использованием базы знаний:\n", result)

Ответ с использованием базы знаний:
 Используй следующие фрагменты контекста, чтобы ответить на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь.

Слабая мотивация труда, приводящая к низкой производительности (зарплата давно не увеличивалась, долгое время нет повышения в должности), порождающей хронический недостаток времени.

В настоящее время существует множество школ тайм-менеджмента. Классическое обучение проводится по следующим темам:

Личная миссия как эмоционально мотивирующий фактор.

оценить задачи соответственно по категориям A, B и C;

Вопрос: Какие методы управления временем существуют?
Полезный ответ: В настоящее момент существует много методов управления временным промежутком, но наиболее эффективными и полезными являются следующие: тайм, планирование, расстановка приоритетов. В этом разделе мы рассмотрим несколько методов, которые помогут тебе научиться использовать тайм и расстановку приоритетности


In [ ]:
# Функция для трассировки работы
def trace_response(query, response):
    print(f"Запрос: {query}")
    print(f"Ответ: {response}")
    print("---")

# Пример запросов для трассировки
queries = [
    "Какие методы управления временем существуют?",
    "Как построить машину времени?",
    "Как я могу повысить свою продуктивность?",
    "Что такое метод Эйзенхауэра?"
]

# Трассировка ответов
for query in queries:
    response = qa_chain.invoke({"query": query})
    trace_response(query, response["result"])

Запрос: Какие методы управления временем существуют?
Ответ: Используй следующие фрагменты контекста, чтобы ответить на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь.

Слабая мотивация труда, приводящая к низкой производительности (зарплата давно не увеличивалась, долгое время нет повышения в должности), порождающей хронический недостаток времени.

В настоящее время существует множество школ тайм-менеджмента. Классическое обучение проводится по следующим темам:

Личная миссия как эмоционально мотивирующий фактор.

оценить задачи соответственно по категориям A, B и C;

Вопрос: Какие методы управления временем существуют?
Полезный ответ: Это — планирование. Планирование — это планирование, а не слепая механическая реакция. Вы должны быть готовы к тому, как использовать время. Не нужно бояться его. Спланируйте каждый день так, будто вы — хозяин собственной жизни. Заплани
---
Запрос: Как построить машину времени?
Ответ: Используй следующие фрагменты контекста, чтобы ответить

In [ ]:
# Тестовый запрос для выявления галлюцинаций
query = "Как построить машину времени?"
response = qa_chain.invoke({"query": query})
trace_response(query, response["result"])

Запрос: Как построить машину времени?
Ответ: Используй следующие фрагменты контекста, чтобы ответить на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь.

Управление рабочим временем

Большой поток рутинных дел не дает менеджеру возможности заниматься основной работой;

Метод «Помидора».

Работа в условиях постоянной спешки приводит к переутомлению.

Вопрос: Как построить машину времени?
Полезный ответ: «Сделать ее из настоящего времени». Этот метод основан на том, как мы используем в своей жизни время. Мы не живем в прошлом, настоящем или будущем, мы живем здесь и сейчас. Когда мы находимся здесь, в настоящем, то не существует никаких
---


In [ ]:
def filter_query(query):
    allowed_topics = ["планирование", "время", "задачи", "приоритеты"]
    if any(topic in query.lower() for topic in allowed_topics):
        return query
    else:
        return "Извините, я не могу ответить на этот вопрос."

# Пример использования фильтра
query = "Как построить машину времени?"
filtered_query = filter_query(query)
response = qa_chain.invoke({"query": filtered_query})
trace_response(query, response["result"])

Запрос: Как построить машину времени?
Ответ: Используй следующие фрагменты контекста, чтобы ответить на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь.

Искусство определять приоритеты.

Из-за чрезмерной занятости менеджер несвоевременно отвечает на деловые письма;

где Пз — потери рабочего времени, зависящие от работника, мин.

Из-за нехватки времени в течение рабочего дня менеджер вынужден завершать работу дома;

Вопрос: Извините, я не могу ответить на этот вопрос.
Полезный ответ: Если бы вы могли ответить мне на один вопрос, то я бы с удовольствием с ним согласился. Я не хочу тратить ваше время на то, чего я знать не знаю. Ответьте на мой вопрос — и я отвечу на ваш. Но это не значит
---


## Потенциальные источники галлюцинаций. Где галлюцинации могут возникнуть?
Недостаток контекста:

Если база знаний не содержит информации по запросу, модель может "придумать" ответ.

Сложные запросы:

Запросы, которые требуют сложных рассуждений или знаний за пределами базы знаний.

Некорректные данные в базе знаний:

Если база знаний содержит ошибки или противоречивую информацию, модель может сгенерировать некорректный ответ.

##Приемы улучшения RAG:
Увеличение базы знаний:

Добавление большего количества релевантной информации.

Оптимизация поиска:

Использование более точных методов поиска релевантных фрагментов.

Улучшение шаблона:

Настройка шаблона для более четкого указания модели, что от неё ожидается.

Ограничение длины ответа:

Ограничение длины ответа помогает избежать неконтролируемых рассуждений.

In [ ]:
# Простая фильтрация запроса
def safe_query(query):
    unsafe_words = ["вреди", "убийство", "порно"]
    if any(word in query.lower() for word in unsafe_words):
        return "Ваш запрос заблокирован."
    return query

# Пример использования фильтра
query = "Как я могу устроить убийство?"
safe_query(query)

'Ваш запрос заблокирован.'